In [8]:
import time
import threading
from collections import deque

In [19]:
class RateLimiter_Method:
    """
        Second (Generic) Rate Limiter class for method rate limits
    """
    def __init__(self, limits):
        """
            Args:
                limits: A list of tuples, where each tuple contains (max_requests, period_in_seconds)
        """
        self.limits = limits
        self.lock = threading.Lock()
        self.request_times = {limit: deque() for limit in self.limits}

    def acquire(self):
        # Only one thread at a time
        with self.lock:
            while True:
                now = time.time()
                allowed = True

                # Go through every request and ensure rates are being followed
                for max_requests, time_window in self.limits:
                    q = self.request_times[(max_requests, time_window)]
                    # Remove timestamps older than the time window
                    while q and q[0] < now - time_window:
                        q.popleft()

                    if len(q) >= max_requests:
                        # Rate limit has been hit, will sleep for this one and then update all rates again / continue checking
                        allowed = False
                        wait_time = time_window - (now - q[0])
                        print(f"Rate limit reached for {max_requests}/{time_window}s. Sleeping for {wait_time:.2f} seconds.")
                        time.sleep(wait_time)
                        break

                if allowed:
                    # If here that means got through every limit check
                    for max_requests, time_window in self.limits:
                        # For every limit, add this timestamp
                        self.request_times[(max_requests, time_window)].append(now)
                    break
                    
            current_time = time.time()
    
            # Find time elapsed since first 
            elapsed = current_time - self.start_time
    
            # Reset calls and start time if period passed
            if elapsed > self.period:
                self.calls = 0
                self.start_time = current_time
    
            # Proceed to sleep or not depending on if max calls per second exceeded
            if self.calls < self.max_calls:
                self.calls += 1
            else:
                time_to_wait = self.period - elapsed
                if time_to_wait > 0:
                    print(f"Rate limit reached for {MAX_REQUESTS_PER_SECOND}/1s. Sleeping for {time_to_wait:.2f} seconds.")
                    time.sleep(time_to_wait)
                self.calls = 1
                self.start_time = time.time()
